In [1]:
import pandas as pd
import numpy as np
import pickle
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from pandas.tools.plotting import scatter_matrix
from pylab import rcParams
import seaborn as sb
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from sklearn.externals import joblib
import datetime

In [2]:
input_folder_path= r"D:\Masters\SJSU\Semester 2\256\Project\GIT\aravinda_regression\Walmart--Search-Optimization-and-recommendations"
filename = "/WalmartDatasset.txt"

search_string_counts = {}
highfreq_searchStrings = []
model_map = {}
search_string_model_map = {}


header_names=["term","product_id", "language","product_impressions","Product_clicks","cart_adds","cart_start","checkout","order"]
essential_features = ['term', 'product_id', 'product_impressions', 'Product_clicks', 'cart_adds', 'checkout', 'order']
columns_all = ['searchTerm', 'product_id', 'product_impressions', 'Product_clicks', 'cart_adds', 'order', 'CTR', 'ATR','conv']
column_scores = ['searchTerm','CTR','ATR','conv','score','ModelName','kfold','pred_ctr']#'Model','Xtest','Ytest']

models = ['Linear','RandomForest']

df = pd.read_csv(input_folder_path+filename, sep='\t', names = header_names)
df_scores = pd.DataFrame(columns=column_scores)

location = 0

In [3]:
def filter_data_by_language_and_features(language,dataframe,feature_list):
    dataframe = dataframe[dataframe['language'] == language]
    return (dataframe[feature_list])
    

In [4]:
def compute_ATR_CTR_Conv(dataframe):
    print('Computing ATR, CTR and Conv...')
    ATR = dataframe['cart_adds'] / dataframe['product_impressions']
    CTR = dataframe['Product_clicks'] / dataframe['product_impressions']
    Conv = dataframe['order'] / dataframe['product_impressions']
    return ATR,CTR,Conv
    

In [5]:
def merge_dataframe_with_computed_values(dataframe_english,columns):
    dataframe_english['searchTerm'] = dataframe_english['term']
    dataframe_english['ATR'],dataframe_english['CTR'],dataframe_english['conv'] = compute_ATR_CTR_Conv(dataframe = dataframe_english)
    print('Merging Dataframe with ATR, CTR and Conv..')
    dataframe_english = dataframe_english[list(columns)].reset_index()
    final_merged_dataset = dataframe_english[list(columns)]
    print('Merging Completed')
    print(final_merged_dataset.info())
    return final_merged_dataset

In [6]:
def replace_NAN_with_zero(dataframe):
    print('Replaced NAN with zeros..')
    return dataframe.replace(np.nan, 0)

In [7]:
def remove_duplicates(dataframe):
    print('Duplicates are Dropped..')
    return dataframe.drop_duplicates()

In [8]:
def calculate_search_string_occurrences(dataframe):
    for i in dataframe['searchTerm']:
        if i in search_string_counts:
            search_string_counts[i]+=1
        else:
            search_string_counts[i]=1

In [9]:
def compute_high_frequency_search_strings(frequency):
    print('Computing number of search strings with occurrences greater than ',frequency)
    for j in search_string_counts:
        if search_string_counts[j] > frequency:
            highfreq_searchStrings.append(j)
    print('Total number of high frequency search strings are ',len(highfreq_searchStrings))
    print(highfreq_searchStrings)

In [10]:
def compute_ATR_CTR_metrics(dataframe,search_string):
    ATRstd,CTRstd,CONVstd = dataframe[['ATR','CTR','conv']].std()
    ATRmean,CTRmean,CONVmean = dataframe[['ATR','CTR','conv']].mean()
    print('ATR mean for search string',search_string,'is',ATRmean)
    print('ATR standard deviation for search string',search_string,'is',ATRstd)
    print('CTR mean for search string',search_string,'is',CTRmean)
    print('CTR standard deviation for search string',search_string,'is',CTRstd)
    return ATRmean,ATRstd,CTRmean,CTRstd,CONVmean,CONVstd

In [11]:
def filter_outliers(dfsearch,ATR_mean,ATR_std,CTR_mean,CTR_std,CONV_mean,CONV_std):
    dfsearchWoOutliers = dfsearch[((dfsearch['ATR']<=(ATR_mean+2*ATR_std))& (dfsearch['ATR']>=(ATR_mean-1*ATR_std)))]
    dfsearchWoOutliers = dfsearchWoOutliers[((dfsearch['CTR']<=(CTR_mean+2*CTR_std))& (dfsearch['CTR']>=(CTR_mean-1*CTR_std)))]
    dfsearchWoOutliers = dfsearchWoOutliers[((dfsearch['conv']<=(CONVmean+2*CONVstd))& (dfsearch['conv']>=(CONVmean-1*CONVstd)))]
    return dfsearchWoOutliers

In [12]:
def plot_3d(dataframe):
    rcParams['figure.figsize'] = 5,5
    sb.set_style('whitegrid')
    #plt.hist(dfsearch['CTR'])
    #plt.plot()
    sb.pairplot(dataframe,vars=['CTR','ATR','conv'], y_vars=['conv'])
    dataframe.plot(kind='scatter',x='ATR',y='conv',c=['darkgray'],s=150)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(dataframe['CTR'], dataframe['ATR'], dataframe['conv'], c='r', marker='o')
    ax.set_xlabel('CTR')
    ax.set_ylabel('ATR')
    ax.set_zlabel('CONV')
    plt.show()


In [13]:
def test_train_split(dataframe,attr1,attr2,testSize):
    return train_test_split(dataframe[[attr1]],dataframe[[attr2]],test_size=int(testSize))

In [14]:
def perform_Regression_Analysis(CTR,ATR,CONV,dataframe):
    if(CTR & ATR & CONV):
        Regression('MultipleLinear')
    for i in models:
        Regression(i)
    

In [15]:
def Linear_Regression(dataframe,attribute1,attribute2,testSize,string):
    regressor = LinearRegression()
    
    if 'CTR' not in [str(attribute1),str(attribute2)]:
        X_train, X_test, y_train, y_test = train_test_split(dataframe[['ATR']],dataframe[['conv']],test_size=0.2)
        x,y = dataframe[['ATR']],dataframe[['conv']]
        regressor.fit(x, y)
        predictedCTR = regressor.predict(x)
       # print('dataframe[searchTerm]',dataframe['searchTerm'])
        print(regressor.score(X_test, y_test))
        scores = cross_val_score(regressor, x, y, cv=10)
        row = [string,False,True,True,regressor.score(X_test, y_test),'Linear_Regression',scores.mean(),predictedCTR]#regressor,X_test,y_test]
        
    elif 'ATR' not in [str(attribute1),str(attribute2)]:
        X_train, X_test, y_train, y_test = train_test_split(dataframe[['CTR']],dataframe[['conv']],test_size=0.2)
        x,y = dataframe[['CTR']],dataframe[['conv']]
        regressor.fit(x, y)
        predictedCTR = regressor.predict(x)
      #  print('dataframe[searchTerm]',dataframe['searchTerm'])
        print(regressor.score(X_test, y_test))
        scores = cross_val_score(regressor, x, y, cv=10)
        row = [string,True,False,True,regressor.score(X_test, y_test),'Linear_Regression',scores.mean(),predictedCTR]#regressor,X_test,y_test]
        
    else:
        X_train, X_test, y_train, y_test = train_test_split(dataframe[['CTR']],dataframe[['ATR']],test_size=0.2)
        x,y = dataframe[['CTR']],dataframe[['ATR']]
        regressor.fit(x, y)
        predictedCTR = regressor.predict(x)
        print('dataframe[searchTerm]',dataframe['searchTerm'])
        print(regressor.score(X_test, y_test))
        scores = cross_val_score(regressor, x, y, cv=10)
        row = [string,True,True,False,regressor.score(X_test, y_test),'Linear_Regression',scores.mean(),predictedCTR]#regressor,X_test,y_test]
        
    location = len(df_scores) +1
    df_scores.loc[location] =row
    search_string_model_map['Linear_Regression'] = regressor
    
    
                                                                                                         

In [16]:
def Multiple_Linear_Regression(dataframe,testSize,string):
    x = dataframe.iloc[:, 6:-1].values
    y = dataframe.iloc[:, 8].values
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
    regressor = LinearRegression()
    regressor.fit(X_train, y_train)
    scores = cross_val_score(regressor, x, y, cv=10)
    predictedCTR = regressor.predict(x)
    row = [string,True,True,True,regressor.score(X_test, y_test),'Multiple_Linear_Regression',scores.mean(),predictedCTR]#regressor,X_test,y_test]   
    location = len(df_scores) +1
    df_scores.loc[location] =row
    search_string_model_map['Multiple_Linear_Regression'] = regressor
    

In [17]:
def Support_Vector_Regression(dataframe,testSize,string):
    X = dataframe.iloc[:, 6:-1].values
    y = dataframe.iloc[:, 8].values
    sc_X = StandardScaler()
    sc_y = StandardScaler()
    X = sc_X.fit_transform(X)
    y = sc_y.fit_transform(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    regressor = SVR(kernel='rbf')
    regressor.fit(X_train, y_train)
    scores = cross_val_score(regressor, X, y, cv=10)
    predictedCTR = regressor.predict(X)
    row = [string,True,True,True,regressor.score(X_test, y_test),'Support_Vector_Regression',scores.mean(),predictedCTR]#regressor,X_test,y_test]   
    location = len(df_scores) +1
    df_scores.loc[location] =row
    search_string_model_map['Support_Vector_Regression'] = regressor
    

In [18]:
def Polynomial_Regression_degree(dataframe,testSize,string,poly_degree):
    features = PolynomialFeatures(degree=poly_degree)
    X_Poly = features.fit_transform(dataframe[['CTR','ATR']])
    y = dataframe[['conv']]
    X_train, X_test, y_train, y_test = train_test_split(X_Poly, y, test_size=0.2, random_state=0)
    regressor = LinearRegression()
    regressor.fit(X_Poly,y)
    scores = cross_val_score(regressor, X_Poly, y, cv=10)
    predictedCTR = regressor.predict(X_Poly)
    row = [string,True,True,True,regressor.score(X_test, y_test),'Polynomial_Regression_degree'+str(poly_degree),scores.mean(),predictedCTR]#regressor,X_test,y_test]   
    location = len(df_scores) +1
    df_scores.loc[location] =row
    search_string_model_map['Polynomial_Regression_degree'+str(poly_degree)] = regressor

In [19]:
def KNN(dataframe,testSize,string,neighbors):
    x = dataframe[['CTR','ATR']]
    y = dataframe[['conv']]
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
    regressor = KNeighborsRegressor(n_neighbors=neighbors)
    regressor.fit(x,y)
    scores = cross_val_score(regressor, x, y, cv=10)
    predictedCTR = regressor.predict(x)
    row = [string,True,True,True,regressor.score(X_test, y_test),'KNN',scores.mean(),predictedCTR]#regressor,X_test,y_test]   
    location = len(df_scores) +1
    df_scores.loc[location] =row
    search_string_model_map['KNN'] = regressor

In [20]:
def RandomForest(dataframe,testSize,string):
    x = dataframe[['CTR']]
    y = dataframe[['conv']]
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
    regressor = RandomForestRegressor()
    regressor.fit(x,y)
    scores = cross_val_score(regressor, x, y, cv=10)
    predictedCTR = regressor.predict(x)
    row = [string,True,False,True,regressor.score(X_test, y_test),'RandomForest',scores.mean(),predictedCTR]#regressor,X_test,y_test]   
    location = len(df_scores) +1
    df_scores.loc[location] =row
    search_string_model_map['RandomForest'] = regressor

In [21]:
def find_best_fit(search_string):
    search_string_scores = df_scores[df_scores.searchTerm == search_string]
    best_fit_score = search_string_scores[search_string_scores.kfold == search_string_scores.kfold.max()]
    best_fit_model = search_string_model_map[((best_fit_score.ModelName).values)[0]]
    predictedCTR = ((best_fit_score.pred_ctr).values)[0]
    return best_fit_model,((best_fit_score.kfold).values)[0], predictedCTR

In [22]:
def save_best_fit_model(best_fit_model,search_string,r2score,recommend):
    modelfilename = search_string+'.sav'
    joblib.dump(best_fit_model, modelfilename)
    model_map[search_string] = [modelfilename,r2score,recommend]
    return modelfilename

In [23]:
def save_model_map():
    now = datetime.datetime.now()
    modelmapfilename = 'savedmodelsmap_'+'.sav'
    joblib.dump(model_map, modelmapfilename)
    return modelmapfilename

In [24]:
def top10Recommendations(search_string,dataframe,model,CTRPred):
    prodmap = {}
    recomm = []
    predictedCTR = CTRPred
    actualCTR = (dataframe.CTR).values
    products = (dataframe.product_id).values
    print(len(predictedCTR))
    print(len(actualCTR))
    for i in range(len(predictedCTR)):
        if(predictedCTR[i] > actualCTR[i]):
            actualCTR[i] = predictedCTR[i]
    for j in range(len(actualCTR)):
        prodmap[products[j]] = actualCTR[j]
    print(len(prodmap))   
    actualCTR.sort()

    for k in range(len(actualCTR)):
        for key, value in prodmap.items():
            if(actualCTR[k] == value):
                recomm.append(key)
    return recomm

    #loaded_model_map = joblib.load('savedmodelsmap_.sav')
    #search_string_model_pat = loaded_model_map[search_string][0]
    #search_string_model = joblib.load(search_string_model_pat)
#    predictedCTR = model.predict((dataframe.CTR).values)
 #   actualCTR = (dataframe.CTR).values
  #  comparray = np.greater(predictedCTR,actualCTR)
   # for i in range(len(comparray)):
    #    if(comparray[i]):
     #       actualCTR[i] = predictedCTR[i]
    #dataframe['AdjCTR'] = actualCTR.tolist()
   # print(dataframe)
    
 



    
    

In [25]:
df_english = filter_data_by_language_and_features(language = 'English',dataframe = df,feature_list = essential_features)
final_merged_dataset = merge_dataframe_with_computed_values (dataframe_english = df_english,columns =columns_all)
final_merged_dataset = replace_NAN_with_zero(final_merged_dataset)
final_merged_dataset = remove_duplicates(final_merged_dataset)
#print(final_merged_dataset.info())

df_for_evaluation = final_merged_dataset[final_merged_dataset['conv'] >0]
calculate_search_string_occurrences(df_for_evaluation)
print('Total number of search strings are', len(search_string_counts))
compute_high_frequency_search_strings(frequency = 50)  

Computing ATR, CTR and Conv...
Merging Dataframe with ATR, CTR and Conv..
Merging Completed
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1330548 entries, 0 to 1330547
Data columns (total 9 columns):
searchTerm             1329176 non-null object
product_id             1330548 non-null int64
product_impressions    1330548 non-null int64
Product_clicks         1330548 non-null int64
cart_adds              1330548 non-null int64
order                  1330548 non-null int64
CTR                    1330548 non-null float64
ATR                    1330548 non-null float64
conv                   1330548 non-null float64
dtypes: float64(3), int64(5), object(1)
memory usage: 91.4+ MB
None
Replaced NAN with zeros..
Duplicates are Dropped..
Total number of search strings are 33982
Computing number of search strings with occurrences greater than  50
Total number of high frequency search strings are  21
[0, 'baby food', 'gluten free', 'food', 'barbie', 'chips', 'camping', 'shampoo', 'paw patrol

In [26]:
highfreq_searchStrings = [x for x in highfreq_searchStrings if not any(c.isdigit() for c in str(x))]
#print(highfreq_searchStrings)
for i in highfreq_searchStrings:
    df_search = df_for_evaluation[df_for_evaluation['searchTerm']==str(i)]
    ATRmean,ATRstd,CTRmean,CTRstd,CONVmean,CONVstd = compute_ATR_CTR_metrics(dataframe = df_search,search_string = str(i))
    dfsearchWoOutliers = filter_outliers(dfsearch = df_search, ATR_mean = ATRmean, ATR_std = ATRstd, CTR_mean = CTRmean, CTR_std = CTRstd,CONV_mean = CONVmean, CONV_std = CONVstd)
    #plot_3d(dfsearchWoOutliers)
    Linear_Regression(dataframe = dfsearchWoOutliers,attribute1 = 'CTR',attribute2 = 'conv',testSize = 0.2,string = str(i))
    Linear_Regression(dataframe = dfsearchWoOutliers,attribute1 = 'ATR',attribute2 = 'conv',testSize = 0.2,string = str(i))
    Multiple_Linear_Regression(dataframe = dfsearchWoOutliers,testSize = 0.2,string = str(i))
    Support_Vector_Regression(dataframe = dfsearchWoOutliers,testSize = 0.2,string = str(i))
    Polynomial_Regression_degree(dataframe = dfsearchWoOutliers,testSize = 0.2,string = str(i),poly_degree = 2)
    Polynomial_Regression_degree(dataframe = dfsearchWoOutliers,testSize = 0.2,string = str(i),poly_degree = 3)
    KNN(dataframe = dfsearchWoOutliers,testSize = 0.2,string = str(i),neighbors = 10)
    RandomForest(dataframe = dfsearchWoOutliers,testSize = 0.2,string = str(i))
    
    model1,r2,predictedCTR = find_best_fit(search_string = str(i))
    recommendations = top10Recommendations(search_string = str(i),dataframe = dfsearchWoOutliers,model = model1,CTRPred = predictedCTR)
    save_best_fit_model(best_fit_model = model1,search_string = str(i),r2score = r2,recommend = recommendations)
    
    
    
save_model_map()

ATR mean for search string baby food is 0.0474781570204
ATR standard deviation for search string baby food is 0.0654319433583
CTR mean for search string baby food is 0.0396878497539
CTR standard deviation for search string baby food is 0.0357953063185
0.846352167859
0.260964338636


C:\Users\Srivatsa\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Srivatsa\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
C:\Users\Srivatsa\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\Srivatsa\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either u

116
116
116
ATR mean for search string gluten free is 0.0331128037838
ATR standard deviation for search string gluten free is 0.0408392597053
CTR mean for search string gluten free is 0.0537090404942
CTR standard deviation for search string gluten free is 0.0554631072552
0.926903579977
-0.544843669687


C:\Users\Srivatsa\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Srivatsa\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
C:\Users\Srivatsa\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\Srivatsa\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either u

66
66
66
ATR mean for search string food is 0.300814740823
ATR standard deviation for search string food is 0.659010879093
CTR mean for search string food is 0.196106683418
CTR standard deviation for search string food is 0.309958037668
0.726917909912
0.493073025662


...

C:\Users\Srivatsa\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Srivatsa\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
C:\Users\Srivatsa\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\Srivatsa\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either u

82
82
82
ATR mean for search string cereal is 0.0400404199633
ATR standard deviation for search string cereal is 0.101056360091
CTR mean for search string cereal is 0.038581045133
CTR standard deviation for search string cereal is 0.0355027030253
0.667268262307
0.203050287298


C:\Users\Srivatsa\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Srivatsa\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
C:\Users\Srivatsa\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\Srivatsa\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either u

73
73
73


'savedmodelsmap_.sav'

In [ ]:
search_string_scores = df_scores[df_scores.searchTerm == 'food']
best_fit_score = search_string_scores[search_string_scores.kfold == search_string_scores.kfold.max()]
best_fit_score.index
#best_fit_model = search_string_model_map[[str(best_fit_score.ModelName)]
#type(best_fit_score)



In [ ]:
def sample(a,b,c):
    if(a & b & c):
        print('true')
        
sample(a = True,b = True, c = True)

In [ ]:
column_scores = ['searchTerm','CTR','ATR','conv','Accuracy','Model']
df_scores = pd.DataFrame(columns=column_scores)
df_scores['searchTerm'] = 'cat'

In [ ]:
(df_scores[(df_scores['kfold']>0.2)])


In [ ]:
len(df_scores[(df_scores['score']>0.5)])

In [ ]:
model = df_scores[(df_scores['searchTerm']=='cereal') & (df_scores['ModelName']=='SupportVectorRegression')]['Model']

In [ ]:
regressor1 = LinearRegression()
regressor2 = RandomForestRegressor()
#type(regressor1)
type(regressor1)

In [ ]:
sample = {}
sample['p'] = [regressor1,1]
sample['q'] = [regressor2,2]

In [ ]:
sample

In [ ]:
type(sample['p'][0])

In [ ]:
model = sample['p'][0]
type(model)

In [ ]:
df_crack = df_scores[df_scores.searchTerm == 'crackers']

df_crack = df_crack[df_crack.kfold == df_crack.kfold.max()]
(float(df_crack.kfold))
#df_crack['kfold'].max()

In [38]:
df_scores[df_scores.kfold>0.2]

,searchTerm,CTR,ATR,conv,score,ModelName,kfold,pred_ctr
1,baby food,True,False,True,0.846352,Linear_Regression,0.314957,"[[0.00205260510798], [0.00526241731535], [0.00..."
3,baby food,True,True,True,0.328848,Multiple_Linear_Regression,0.285558,"[0.00166403012083, 0.00656209618718, 0.0005502..."
4,baby food,True,True,True,0.587663,Support_Vector_Regression,0.395693,"[-0.552468021413, -0.183049969644, -0.56622194..."
5,baby food,True,True,True,0.611421,Polynomial_Regression_degree2,0.295763,"[[0.00248965875273], [0.00469306446253], [0.00..."
6,baby food,True,True,True,0.581820,Polynomial_Regression_degree3,0.267908,"[[0.00169088128209], [0.00488181237548], [0.00..."
7,baby food,True,True,True,0.539174,KNN,0.293354,"[[0.00166536651174], [0.00505930077426], [0.00..."
15,gluten free,True,True,True,0.223838,KNN,0.470994,"[[0.00104423260026], [0.00142978179207], [0.00..."
17,food,True,False,True,0.726918,Linear_Regression,0.518711,"[[0.412780868022], [0.328050677747], [0.000457..."
21,food,True,True,True,0.942907,Polynomial_Regression_degree2,0.437134,"[[0.409668999832], [0.320468826952], [-0.00142..."
23,food,True,True,True,0.793147,KNN,0.492997,"[[0.288333333333], [0.276025641026], [0.001017..."


In [ ]:
import datetime

now = datetime.datetime.now()
type(now.strftime("%Y-%m-%d %H:%M %S"))

In [ ]:
type(now)

In [ ]:
str(now)

In [ ]:
search_string_model_map

In [ ]:
from sklearn.externals import joblib

loaded_model = joblib.load('savedmodelsmap_.sav')

In [ ]:
search_string_model_pat = loaded_model['great value'][0]
search_string_model = joblib.load(search_string_model_pat)

In [ ]:
now = datetime.datetime.now()
modelmapfilename = 'savedmodels_'+str(now.strftime("%Y-%m-%d-%H:%M"))+'.sav'
joblib.dump(model_map, modelmapfilename)

In [ ]:
(loaded_model['great value'])

In [ ]:
#def top10Recommendations(search_string,dataframe,model):
    #loaded_model_map = joblib.load('savedmodelsmap_.sav')
    #search_string_model_pat = loaded_model_map[search_string][0]
    #search_string_model = joblib.load(search_string_model_pat)
prodmap = {}
recomm = []
model = LinearRegression()
model.fit(dfsearchWoOutliers[['CTR']],dfsearchWoOutliers[['conv']])
predictedCTR = model.predict(dfsearchWoOutliers[['CTR']])
actualCTR = (dfsearchWoOutliers.CTR).values
products = (dfsearchWoOutliers.product_id).values
print(len(predictedCTR))
print(len(actualCTR))
for i in range(len(predictedCTR)):
    if(predictedCTR[i] > actualCTR[i]):
        actualCTR[i] = predictedCTR[i]
for j in range(len(actualCTR)):
    prodmap[products[j]] = actualCTR[j]
print(len(prodmap))   
actualCTR.sort()

for k in range(len(actualCTR)):
    for key, value in prodmap.items():
        if(actualCTR[k] == value):
            recomm.append(key)
(recomm)
 


    
    
    
    
 #   model = LinearRegression()
#    model.fit(dfsearchWoOutliers[['CTR']],dfsearchWoOutliers[['conv']])

In [ ]:
comparray = np.greater(predictedCTR,actualCTR)
for i in range(len(comparray)):
    if(comparray[i]):
        actualCTR[i] = predictedCTR[i]
dfsearchWoOutliers['AdjCTR'] = actualCTR.tolist()
print(dfsearchWoOutliers)

In [29]:
model_map['baby food'][2]

[6000188129415,
 6000032324715,
 6000193235142,
 6000106850445,
 6000192773025,
 6000001830696,
 6000001831109,
 6000193235154,
 6000193235151,
 6000032324671,
 6000001831107,
 6000106851325,
 6000193235131,
 6000192504396,
 6000192504378,
 6000001835291,
 6000001831130,
 6000191920991,
 6000106850920,
 6000188968080,
 6000192504381,
 6000001830190,
 6000001831136,
 6000187770368,
 6000195518209,
 6000032324683,
 1255040140403,
 6000001830625,
 6000106851252,
 6000188129409,
 6000195285369,
 6000001831032,
 6000193235147,
 6000001830698,
 6000195285359,
 6000192504384,
 6000001830189,
 6000001831118,
 6000192504369,
 6000195285346,
 6000135442588,
 6000001830258,
 6000192504363,
 6000192504375,
 6000134546149,
 6000001830185,
 6000189944777,
 6000187770371,
 6000032324685,
 6000195445208,
 6000195527603,
 6000190449014,
 6000001830618,
 6000188129421,
 6000192504402,
 6000195285328,
 6000106851083,
 6000193770053,
 6000001831056,
 6000001830264,
 6000188129424,
 6000188129430,
 6000196